In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob
import zipfile

In [3]:
from google.colab  import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/PCOS_detection_using_DL/PCOS.zip

Archive:  /content/drive/MyDrive/PCOS_detection_using_DL/PCOS.zip
replace PCOS/infected/img_0_1023.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
ROOT_DIR = '/content/PCOS'
number_of_images = {}

for dir in os.listdir(ROOT_DIR):
   number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR,dir)))
   print("",dir,"" ,number_of_images[dir])




In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input

In [ ]:
def preprocessingImage1(path):
  image_data = ImageDataGenerator(zoom_range=0.2,shear_range=0.2,preprocessing_function= preprocess_input,horizontal_flip=True)
  image = image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')
  return image

In [ ]:
def preprocessionfImage2(path):
  """
  Input :path
  Output : preprocessed Image
  """
  image_data  = ImageDataGenerator(preprocessing_function= preprocess_input )
  image = image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')

  return image




We will split the data such that
70% for training
15 % for vailadation
15 % for testing



In [ ]:
def datafolder(path,split):
  if not os.path.exists("./"+path):
    os.mkdir("./"+path)

    for dir in os.listdir(ROOT_DIR):
      os.makedirs("./"+path+"/"+dir)
      for img in np.random.choice(a=os.listdir(os.path.join(ROOT_DIR,dir)),
                                  size=(math.floor(split * number_of_images[dir])-5),replace=False):

          O = os.path.join(ROOT_DIR,dir,img)
          D = os.path.join("./"+path,dir)
          shutil.copy(O,D)
          os.remove(O)

  else:
     print("Folder already exist")


In [ ]:
datafolder("train",0.7)

In [ ]:
datafolder("val",0.15)

In [ ]:
datafolder("test",0.15)

In [ ]:
path ='/content/train'
train_data = preprocessingImage1(path)

In [ ]:
path ='/content/test'
test_data = preprocessionfImage2(path)

In [ ]:
path = '/content/val'
val_data = preprocessionfImage2(path)



Model Block




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten,Dense
from keras.models import Model,load_model
from keras.applications.mobilenet import MobileNet
import keras

In [ ]:
base_model = MobileNet(input_shape=(224,224,3),include_top=False)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
x= Flatten()(base_model.output)
x= Dense(units=1,activation='sigmoid')(x)

In [ ]:
model = Model(base_model.input,x)

In [ ]:
model.compile(optimizer='rmsprop',loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

mc = ModelCheckpoint(filepath="bestmodel.h5",monitor='val_accuracy',verbose=1,save_best_only=True)

#Early check points
es = EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=5,verbose=1)

cb = [mc,es]

In [ ]:
hist = model.fit_generator(train_data,
                           steps_per_epoch=10,
                           epochs=30,
                           validation_data=val_data,
                           validation_steps=16,
                          callbacks=cb)

In [ ]:
model = load_model("/content/bestmodel.h5")

In [41]:
acc = model.evaluate_generator(test_data)[1]
print(f"our model accuracy is  {acc * 100} %")

<ipython-input-41-fe794b425aa5>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc = model.evaluate_generator(test_data)[1]


our model accuracy is  99.64028596878052 %


 Our model is perform well on the datasets.

The accuracy is 100.0 %

In [ ]:
h = hist.history
h.keys()

In [ ]:
train_data.class_indices

In [ ]:
test_data.class_indices

In [ ]:
val_data.class_indices

In [40]:
#now get some random images and predict the class
from keras.preprocessing import image
import tensorflow as tf


def predictimage(path):

      img = tf.keras.utils.load_img(path,target_size=(224,224))
      i = tf.keras.utils.img_to_array(img)/255
      input_arr= np.array([i])
      input_arr.shape

      pred =model.predict(input_arr)
      if pred == 1:
          print("Not Affected")
      else :
         print("Affected")
      #display image
      plt.imshow(input_arr[0],vmin=0, vmax=255)
      plt.title("input Image")
      plt.show()

In [ ]:
#It is the infected image sample
#predictimage("/content/PCOS/infected/img_0_113.jpg")

In [ ]:
#It is the not affected sample images
#predictimage("/content/PCOS/notinfected/img_0_6858.jpg")

In [ ]:
#not infected image

#predictimage("/content/PCOS/notinfected/img_0_1772.jpg")

In [ ]:
#predictimage("/content/PCOS/infected/img_0_9985.jpg")